In [1]:
from fastai.tabular import * 
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import multilabel_confusion_matrix

%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

/Users/Ann/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
crime_property_light_population = pd.read_csv("./model_data/crime_light_density.csv").iloc[:,1:]
crime_property_light_population.head()

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,zipcode,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
0,Auto Theft,E18,N,2019-10-13 09:28:24,2019,10,Sunday,9,LINCOLN ST,42.259518,-71.121563,2136,6206.9,455.644447,21.073507,0.437849
1,Auto Theft,E18,N,2019-10-12 20:11:26,2019,10,Saturday,20,METROPOLITAN AVE,42.262092,-71.116710,2136,6206.9,455.644447,21.073507,0.437849
2,Auto Theft,D14,N,2019-10-12 15:12:43,2019,10,Saturday,15,ALLSTON ST,42.352375,-71.135096,2134,16212.0,2250.588503,63.554783,0.582893
3,Auto Theft,D4,N,2019-10-12 04:41:52,2019,10,Saturday,4,SAINT JAMES AVE,42.349476,-71.076402,2116,32724.1,14799.401260,480.463801,0.656585
4,Auto Theft,A15,N,2019-10-02 08:08:49,2019,10,Wednesday,8,N MEAD ST,42.381846,-71.066551,2129,12192.0,278.377510,165.249738,0.678941


In [3]:
predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density']
df = crime_property_light_population[predictors]
df = df.dropna()
df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
0,Auto Theft,N,10,Sunday,9,6206.9,455.644447,21.073507,0.437849
1,Auto Theft,N,10,Saturday,20,6206.9,455.644447,21.073507,0.437849
2,Auto Theft,N,10,Saturday,15,16212.0,2250.588503,63.554783,0.582893
3,Auto Theft,N,10,Saturday,4,32724.1,14799.401260,480.463801,0.656585
4,Auto Theft,N,10,Wednesday,8,12192.0,278.377510,165.249738,0.678941


In [4]:
top5 = df['OFFENSE_CODE_GROUP'].value_counts().index[:5]

#cut the df
df = df[df['OFFENSE_CODE_GROUP'].isin(top5)]
enc = OneHotEncoder(handle_unknown='ignore')

le = LabelEncoder() 
  
df['SHOOTING']= le.fit_transform(df['SHOOTING'])
df['DAY_OF_WEEK']= le.fit_transform(df['DAY_OF_WEEK']) 
df['OFFENSE_CODE_GROUP'] = le.fit_transform(df['OFFENSE_CODE_GROUP'])

In [5]:
#df_cat = enc.fit_transform(df[["MONTH","SHOOTING","DAY_OF_WEEK"]]).toarray()

#df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['MONTH','SHOOTING', 'DAY_OF_WEEK']))
#df = pd.concat([df, df_cat], axis=1)
#df = df.dropna()

df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
6,0,0,9,3,3,26693.9,3362.723369,581.882742,0.699176
9,3,0,9,3,5,24074.6,5490.169358,260.905478,0.728900
10,0,0,9,3,1,8352.4,2009.317606,59.185114,0.525616
16,3,0,9,3,0,15913.2,1293.230035,31.015855,0.568578
18,0,0,9,3,3,14884.4,1040.984096,31.260815,0.640733


In [6]:
len(df)

147858

In [7]:
procs = [FillMissing, Categorify, Normalize]

In [12]:
valid_idx = range(len(df)-10000, len(df))

In [13]:
dep_var = 'OFFENSE_CODE_GROUP'
cat_names = ["MONTH","SHOOTING","DAY_OF_WEEK"]

data = TabularDataBunch.from_df('./model_data', df, dep_var, \
                                valid_idx=valid_idx, procs=procs, \
                                cat_names=cat_names)

In [14]:
print(data.train_ds.cont_names) 

['LAND_VAL', 'Population density (per square mile of land area)', 'light_density', 'HOUR', 'BLDG_VAL']


In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.530213,1.535988,0.345800,00:14
1,1.535377,1.532450,0.347400,00:15
2,1.531061,1.531480,0.345500,00:16
